# hw 5


Студент Чешев А. Д.<br>




In [1]:
// 1. Какие плюсы и недостатки у Merge Sort Join в отличии от Hash Join? (1 балл)
//
// Метод Merge Sort Join эффективен в случае больших объемов данных, поскольку не требует их полной загрузки в память.
// Его рационально использовать в тех случаях, где данные не могут быть хэшированы равномерно. 
// Однако Merge Sort Join требует предварительной сортировки данных, что может потребовать времени и ресурсов. 
// Этот метод может оказаться менее эффективным для небольших объемов данных.
//
// В свою очередь, Hash Join использует хеш-функции для соотнесения данных между таблицами. 
// Этот метод не требует предварительной сортировки данных и может быть эффективен в случае равномерного распределения
// данных по хеш-функции. 
// Минус данного метода заключается в большей чувствительности к неравномерности данных, что может требовать больше
// оперативной памяти при хешировании данных.

In [2]:
// 2. Соберите WordCount приложение, запустите на датасете ppkm_sentiment (1 балл)

import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SparkSession, DataFrame}

val sc = spark.sparkContext

val wordCountSpark = SparkSession.builder().appName("WordCount").config(sc.getConf).getOrCreate()
val filePath = "datasets/ppkmSentiment/ppkm_dataset.csv"

val reviewsDf: DataFrame = wordCountSpark.read
        .option("header", true)
        .option("escape", "\"")
        .schema("text STRING, label INT")
        .csv(filePath)

val wcDf: DataFrame = reviewsDf
        .select(explode(split($"text", raw"\W+")).as("word"))
        .where($"word" =!= "")
        .groupBy($"word")
        .count()
        .orderBy($"count".desc)

wcDf.show()

+-------+-----+
|   word|count|
+-------+-----+
|negatif|  100|
|positif|  100|
| netral|  100|
+-------+-----+



In [3]:
// 3. Измените WordCount так, чтобы он удалял знаки препинания и приводил все слова к единому регистру (1 балл)

var punktLowerDf: DataFrame = wordCountSpark.read
        .option("header", true)
        .option("escape", "\"")
        .schema("text STRING, label STRING")
        .csv(filePath)
        .withColumn("label", lower(regexp_replace($"label", "[\\p{Punct}]", "")))

punktLowerDf.show()

+-------+--------------------+
|   text|               label|
+-------+--------------------+
|positif|kami siap laksana...|
|positif|siap melaksanakan...|
|positif|siap dukung dan s...|
|positif|langkah 3m ini su...|
|positif|siap amankan selu...|
|positif|siap utk di sosia...|
|positif|mendukung kebijak...|
|positif|mari bersama cega...|
|positif|mari kita suksesk...|
|positif|siap kawal dan am...|
|positif|semoga pak gubern...|
|positif|semoga bapak sela...|
|positif|     semangat selalu|
|positif|semoga berhasilmi...|
|positif|patuhi prokes dan...|
|positif|terimakasih atas ...|
|positif|sehat selalu pemi...|
|positif|semoga selalu dib...|
|positif|semoga pandemi se...|
|positif|makasih bu sukses...|
+-------+--------------------+
only showing top 20 rows



In [4]:
// 4. Измените выход WordCount так, чтобы сортировка была по количеству повторений, а список слов был во
//    втором столбце, а не в первом (1 балл)

val reviewsDf: DataFrame = wordCountSpark.read
        .option("header", true)
        .option("escape", "\"")
        .schema("text STRING, label INT")
        .csv(filePath)

val wcDf: DataFrame = reviewsDf
    .select(explode(split($"text", raw"\W+")).as("word"))
    .where($"word" =!= "")
    .groupBy($"word")
    .count()
    .orderBy($"count".desc)
    .select($"count", $"word")

wcDf.show()

+-----+-------+
|count|   word|
+-----+-------+
|  100|negatif|
|  100|positif|
|  100| netral|
+-----+-------+



In [5]:
// 5. Измените выход WordCount, чтобы формат соответствовал TSV (1 балл)

val reviewsDf: DataFrame = wordCountSpark.read
        .option("header", true)
        .option("escape", "\"")
        .schema("text STRING, label INT")
        .csv(filePath)

val wcDf: DataFrame = reviewsDf
        .select(explode(split($"text", raw"\W+")).as("word"))
        .where($"word" =!= "")
        .groupBy($"word")
        .count()
        .orderBy($"count".desc)
        .select(concat_ws("\t", $"word", $"count").alias("word_count"))

wcDf.show()

+-----------+
| word_count|
+-----------+
|negatif	100|
|positif	100|
| netral	100|
+-----------+



In [6]:
// 6. Добавьте в WordCount возможность через конфигурацию задать список стоп-слов, которые будут 
//    отфильтрованы во время работы приложения (2 балла)
val stopWordsList = wordCountSpark.conf.get("spark.wordcount.stopwords", "").split(",").map(_.trim)

val reviewsDf: DataFrame = wordCountSpark.read
        .option("header", true)
        .option("escape", "\"")
        .schema("text STRING, label INT")
        .csv(filePath)

// В данном случае исключили слово "negatif".
val wcDf: DataFrame = reviewsDf
        .select(explode(split($"text", raw"\W+")).as("word"))
        .where(!($"word".isin(stopWordsList: _*)))
        .groupBy($"word")
        .count()
        .orderBy($"count".desc)

wcDf.show()

+-------+-----+
|   word|count|
+-------+-----+
|positif|  100|
| netral|  100|
+-------+-----+



In [7]:
// 7. Выполните broadcast join на двух датасетах, не используя метод join(). Можно использовать любые предварительные
//    трансформации. В качестве исходных данных возьмите Company.csv и Company_Tweet.csv из датасета Tweets about
//    the Top Companies from 2005 to 2020 (3 балла)


// Решение не соответствует теребованиям.
// 
// val filePath1 = "datasets/TopCompaniesTweets/Company.csv"
// val filePath2 = "datasets/TopCompaniesTweets/Company_Tweet.csv"
// val companyDF: DataFrame = wordCountSpark.read.option("header", true).csv(filePath1)
// val tweetDF: DataFrame = wordCountSpark.read.option("header", true).csv(filePath2)
//
// companyDF.show()
//
// tweetDF.show()
//
// val resultDF = tweetDF.join(broadcast(companyDF), "ticker_symbol")
//
// resultDF.show()

+-------------+------------+
|ticker_symbol|company_name|
+-------------+------------+
|         AAPL|       apple|
|         GOOG|  Google Inc|
|        GOOGL|  Google Inc|
|         AMZN|  Amazon.com|
|         TSLA|   Tesla Inc|
|         MSFT|   Microsoft|
+-------------+------------+

+------------------+-------------+
|          tweet_id|ticker_symbol|
+------------------+-------------+
|550803612197457920|         AAPL|
|550803610825928706|         AAPL|
|550803225113157632|         AAPL|
|550802957370159104|         AAPL|
|550802855129382912|         AAPL|
|550802745737768960|         AAPL|
|550797494188142592|         AAPL|
|550797275786518528|         AAPL|
|550797272686923776|         AAPL|
|550796617444765696|         AAPL|
|550795512899960832|         AAPL|
|550795254102638593|         AAPL|
|550795167318700033|         AAPL|
|550795088821886976|         AAPL|
|550793298357391360|         AAPL|
|550793247669231616|         AAPL|
|550793108242198528|         AAPL|
|55079191